# Max sortable chunks
Determine the maximum number of chunks an array can be broken into and then individually sorted that will result in a sort of the original array when concatenated. 

In [89]:
import unittest
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

## First Strategy
The maximum possible number of chunks i n, where n is the length of the array. If the array is already sorted this will be the answer. Otherwise for any given chunk in the array it should be merged to the right if its min is less than the max of everything to the right, and merged to the left if its min is less than the max of the left. We start with n chunks and work from right to left successively merging when needed

### Why it is wrong or slow
After each successful merge the new merged partition needs to be compared to each piece to the right of it to make sure we didnt miss an earlier merge, hence why it fails the 3rd test case, `[4,0,0,2,4]`

In [90]:
class Solution:
    def print_partitions(self, arr, partitions):
        print([arr[p[0]:p[1]] for p in partitions])
    
    def maxChunksToSorted(self, arr) -> int:
        # start with as many chunks as there are elements each of length 1
        partitions = [(i, i+1) for i in range(len(arr))]
        cursor = len(arr) - 2
        print(partitions)
        while cursor >= 0:
            p = partitions[cursor]
            pright = partitions[cursor + 1]
            pleft = partitions[cursor - 1]
            min_current = min(arr[p[0]:p[1]])
            max_current = max(arr[p[0]:p[1]])
            max_left = max(arr[pleft[0]:pleft[1]])
            min_right = min(arr[pright[0]:pright[1]])
            print("curr, minc, maxc, maxl, minr")
            print("{:4d}, {:4d}, {:4d}, {:4d}, {:4d}".format(cursor, min_current, max_current, max_left, min_right))
            self.print_partitions(arr, partitions)
            if min_current < max_left and cursor > 0:
                # current partition minimum less than left side maximum, merge current into left
                new = (pleft[0],p[1])
                partitions[cursor] = new
                partitions.pop(cursor - 1)
                cursor -= 1
            
            elif max_current > min_right:
                # current partition maximum greater than right side minimum
                pnew = (p[0], pright[1])
                partitions[cursor] = pnew
                partitions.pop(cursor + 1)
                cursor -= 1
                
            else: 
                cursor -= 1
                
            print(partitions)

        self.print_partitions(arr, partitions)
        return len(partitions)
                

In [91]:
# test 1, a reverse sorted list has 1 chunk
testlist = [5, 4, 3, 2, 1]
print(testlist)
s = Solution()
s.maxChunksToSorted(testlist)

[5, 4, 3, 2, 1]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   3,    2,    2,    3,    1
[[5], [4], [3], [2], [1]]
[(0, 1), (1, 2), (2, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   2,    2,    3,    4,    1
[[5], [4], [3, 2], [1]]
[(0, 1), (1, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   1,    2,    4,    5,    1
[[5], [4, 3, 2], [1]]
[(0, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   0,    2,    5,    1,    1
[[5, 4, 3, 2], [1]]
[(0, 5)]
[[5, 4, 3, 2, 1]]


1

In [92]:
# test 2, a sorted list as n chunks
testlist = [1, 2, 3, 4, 5]
s = Solution()
s.maxChunksToSorted(testlist)

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   3,    4,    4,    3,    5
[[1], [2], [3], [4], [5]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   2,    3,    3,    2,    4
[[1], [2], [3], [4], [5]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   1,    2,    2,    1,    3
[[1], [2], [3], [4], [5]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   0,    1,    1,    5,    2
[[1], [2], [3], [4], [5]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
[[1], [2], [3], [4], [5]]


5

In [93]:
# test 3, a little more complex, has 2 chunks

testlist = [4,0,0,2,4]
s = Solution()
s.maxChunksToSorted(testlist)

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   3,    2,    2,    0,    4
[[4], [0], [0], [2], [4]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   2,    0,    0,    0,    2
[[4], [0], [0], [2], [4]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   1,    0,    0,    4,    0
[[4], [0], [0], [2], [4]]
[(0, 2), (2, 3), (3, 4), (4, 5)]
curr, minc, maxc, maxl, minr
   0,    0,    4,    4,    0
[[4, 0], [0], [2], [4]]
[(0, 3), (3, 4), (4, 5)]
[[4, 0, 0], [2], [4]]


3

## Second Strategy
We take a greedy aproach, in an array $A$ of $p$ elements we if $[a_i:a_m]$ is a valid chunk and $[a_i:a_n]$ is a valid chunk, $n > m$ then $[a_m:a_n]$ is a valid chunk. We therefore move from left to right and at each place evaluate if the current chunk ios valid. A selection is a valid chunk if its maximum is less than the minimum of everything to its right.

In [102]:
class Solution:
    def maxChunksToSorted(self, arr) -> int:
        chunks = 1
        n = len(arr)
        for i in range(n):
            test_chunk = arr[0:i]
            rem = arr[i:n]
            print(test_chunk, rem)
            if test_chunk and rem and max(test_chunk) <= min(rem):
                chunks += 1
        return chunks


In [103]:
testlist = [4,0,0,2,4]
s = Solution()
s.maxChunksToSorted(testlist)

[] [4, 0, 0, 2, 4]
[4] [0, 0, 2, 4]
[4, 0] [0, 2, 4]
[4, 0, 0] [2, 4]
[4, 0, 0, 2] [4]


2